In [144]:
import os
import sys
import numpy as np
import inspect
import pandas as pd
from pathlib import Path
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font, NamedStyle, Border, PatternFill, Side
from openpyxl.utils import get_column_letter
from openpyxl.utils.dataframe import dataframe_to_rows
import tkinter as tk
from tkinter import filedialog, messagebox

In [145]:

"""error check module"""
class ErrCode:
    """标准错误码常量 (符合 0=成功 约定)"""
    SUCCESS = 0             # 操作成功
    INVALID_ARGUMENT = 1    # 无效参数
    FILE_NOT_FOUND = 2      # 文件不存在
    UNKNOWN_ERROR = 99      # 未知错误

def errcheck(result):
    frame = inspect.currentframe()
    try:
        # 获取调用栈中上一帧（调用此函数的帧）
        caller_frame = frame.f_back
        line =  caller_frame.f_lineno
    finally:
        # 显式删除帧引用以避免内存泄漏
        del frame
    if result != ErrCode.SUCCESS:
        print(f"err_code:{result},line:{line}")


In [146]:

def find_excel_file():
    #测试用
    #=====================================================
    file_path = "/Volumes/SSD 1TB/GEhealthcare/202508/Preventive-Pending Report.xlsx"
    return file_path
    #=====================================================

    #正式用
    """使用 tkinter 选择文件"""
    root = tk.Tk()
    root.withdraw()  # 隐藏主窗口

    # 创建文件选择对话框
    file_path = filedialog.askopenfilename(
        title="选择文件",
        filetypes=[
            ("Excel 文件", "*.xlsx *.xls"),
            ("CSV 文件", "*.csv"),
            ("所有文件", "*.*")
        ],
        initialdir="/Volumes/SSD 1TB/GEhealthcare/"  # 初始目录，可选
    )
    root.destroy()  # 关闭 tkinter 窗口
    if not file_path:
        print("未选择文件，操作取消")
        return None
    return file_path

In [147]:
def save_with_formatting(df, file_path):
    """
    保存DataFrame到Excel文件并应用格式美化
    1. 自动调整列宽
    2. 设置文本自动换行
    3. 冻结首行
    4. 设置标题行加粗
    5. 添加排序提示
    """
    try:
        # 先保存到Excel文件
        df.to_excel(file_path, index=False)

        # 使用openpyxl加载工作簿进行格式设置
        wb = openpyxl.load_workbook(file_path)
        ws = wb.active

        # 设置标题行加粗和自动换行
        for cell in ws[1]:
            cell.font = Font(bold=True)
            cell.alignment = Alignment(wrap_text=True, vertical='top')

        # 设置数据行自动换行和对齐
        for row_idx in range(2, ws.max_row + 1):
            for cell in ws[row_idx]:
                cell.alignment = Alignment(wrap_text=True, vertical='top')

        # 自动调整列宽
        for col_idx in range(1, ws.max_column + 1):
            max_length = 0
            col_letter = openpyxl.utils.get_column_letter(col_idx)

            # 计算每列的最大宽度
            for cell in ws[col_letter]:
                try:
                    value = str(cell.value) if cell.value is not None else ""
                    if len(value) > max_length:
                        max_length = len(value)
                except:
                    pass

            # 设置列宽，限制最大宽度为50
            adjusted_width = min(max_length + 2, 50)
            ws.column_dimensions[col_letter].width = adjusted_width

        # 冻结首行
        ws.freeze_panes = 'A2'

        # 添加排序提示到第一行
        if "Location" in df.columns and "Model" in df.columns:
            # 获取GE_Asset_Location和Model列的索引
            location_idx = list(df.columns).index("Location") + 1
            model_idx = list(df.columns).index("Model") + 1

            # 添加排序提示
            ws.cell(row=1, column=location_idx).value = "Location ▼"
            ws.cell(row=1, column=model_idx).value = "Model ▼"

            # 设置排序提示样式
            ws.cell(row=1, column=location_idx).font = Font(bold=True, color="0070C0")
            ws.cell(row=1, column=model_idx).font = Font(bold=True, color="0070C0")

        # 保存
        wb.save(file_path)
        return True

    except Exception as e:
        print(f"格式化文件时出错: {e}")
        # 如果格式化失败，至少保存原始数据
        df.to_excel(file_path, index=False)
        return False


In [148]:
input_path = '/Volumes/SSD 1TB/GEhealthcare/Doc'
output_path = '/Volumes/SSD 1TB/GEhealthcare/202508/Output'


#通过字典来筛选出需要的列再生成到对应文件夹里
def preserve_format_write(input_path, output_path, data_dict, sheet_name='report_demo'):
    """
    保留原始 Excel 格式的同时写入新数据

    参数:
    input_path: 原始 Excel 文件路径
    output_path: 输出文件路径
    data_dict: 要写入的数据字典 {列字母: 数据列表}
    sheet_name: 工作表名称
    """
    try:
        # 1. 加载原始工作簿，保留所有格式
        wb = load_workbook(input_path)

        # 2. 获取目标工作表
        if sheet_name not in wb.sheetnames:
            raise ValueError(f"工作表 '{sheet_name}' 不存在")

        ws = wb[sheet_name]

        # 3. 备份原始样式
        style_backup = {}
        for col_letter in data_dict.keys():
            col_idx = openpyxl.utils.column_index_from_string(col_letter)
            for row_idx in range(1, ws.max_row + 1):
                cell = ws.cell(row=row_idx, column=col_idx)
                style_backup[(row_idx, col_idx)] = {
                    'font': cell.font.copy(),
                    'border': cell.border.copy(),
                    'fill': cell.fill.copy(),
                    'alignment': cell.alignment.copy(),
                    'number_format': cell.number_format
                }

        # 4. 写入新数据
        for col_letter, values in data_dict.items():
            col_idx = openpyxl.utils.column_index_from_string(col_letter)

            # 确定起始行（跳过标题行）
            start_row = 2

            for i, value in enumerate(values):
                row_idx = start_row + i
                if row_idx > ws.max_row:
                    # 添加新行并复制样式
                    ws.append([None] * ws.max_column)
                    # 复制上一行的样式
                    for col in range(1, ws.max_column + 1):
                        prev_cell = ws.cell(row=row_idx-1, column=col)
                        new_cell = ws.cell(row=row_idx, column=col)
                        new_cell.font = prev_cell.font.copy()
                        new_cell.border = prev_cell.border.copy()
                        new_cell.fill = prev_cell.fill.copy()
                        new_cell.alignment = prev_cell.alignment.copy()
                        new_cell.number_format = prev_cell.number_format

                cell = ws.cell(row=row_idx, column=col_idx)

                # 保留原始数据类型
                if isinstance(value, (int, float, np.integer, np.floating)):
                    cell.value = value
                    if isinstance(value, float):
                        cell.number_format = '0.00'
                else:
                    cell.value = str(value) if value is not None else ""

        # 5. 恢复原始样式
        for (row_idx, col_idx), styles in style_backup.items():
            if row_idx <= ws.max_row and col_idx <= ws.max_column:
                cell = ws.cell(row=row_idx, column=col_idx)
                cell.font = styles['font']
                cell.border = styles['border']
                cell.fill = styles['fill']
                cell.alignment = styles['alignment']
                cell.number_format = styles['number_format']

        # 6. 保存工作簿
        wb.save(output_path)
        print(f"数据已成功写入 {output_path}")
        return True

    except Exception as e:
        print(f"写入数据时出错: {str(e)}")
        return False


In [149]:
def preprocess(file_path):

    """处理数据：选择所需列并清理数据"""
    # 确定需要的列（根据表头）
    required_columns = {
        'Asset ID': 'D',  # 资产ID
        'Location': 'K',  # 位置
        'Manufacture': 'L',  # 制造商
        'Model': 'M',  # 型号
        'Serial No':'N', #SN号
        'Description':'O', #型号描述
        'ZT': 'EV' , # ZT 状态
        'HA Work Order No':'R', #HA Order number
        'Service Report Reference': 'U' ,#GE 参考report
        'Caller':'I', #联系人
        'Caller Tel':'J' #联系电话
        #如果需要可以继续加
    }
    """
    实现预处理:
    1、根据excel文件内容找到关键信息,根据Location分组保存新的独立excel文件
    2、生成设备型号汇总表
    """
    #获取当前文件的目录
    output_dir = Path(file_path).parent

    # 创建输出目录
    output_dir = output_dir/'Output'
    print(output_dir)

    #生成目录
    output_dir.mkdir(parents=True, exist_ok=True)

    try:
        print(f"正在读取文件: {file_path}")
        df = pd.read_excel(file_path, engine='openpyxl')
        print(f"成功读取文件: 共 {len(df)} 条记录")

    except Exception as e:
        print(f"读取文件失败: {e}")
        return ErrCode.FILE_NOT_FOUND


    # 标准化列名（去除空格和大小写差异）
    df.columns = [col.strip().replace(' ', '_') for col in df.columns]

    # 检查必要列是否存在
    required_columns = ['Asset_ID','Location', 'Model']
    missing_columns = [col for col in required_columns if col not in df.columns]

    if missing_columns:
        print(f"错误: 缺少必要列 - {', '.join(missing_columns)}")
        print("请检查Excel文件是否正确:")

    # 获取所有唯一的位置前缀
    unique_prefixes = [p for p in df['Location'].unique() if pd.notna(p)]
    print(f"找到 {len(unique_prefixes)} 个不同的位置")

    # 保存分组结果到单独文件
    for prefix in unique_prefixes:
        # 过滤出当前前缀的数据
        group_df = df[df['Location'] == prefix].copy()

        # 按照Location和Model排序
        group_df = group_df.sort_values(
            by=['Location', 'Model', 'Asset_ID'],
            ascending=[True, True, True]
        ).reset_index(drop=True)

      # 生成安全的文件名
        safe_prefix = str(prefix).replace('/', '_').replace('\\', '_')
        output_path = output_dir / f'{safe_prefix}.xlsx'

        # 保存到Excel并应用格式美化
        save_with_formatting(group_df, output_path)
        print(f"已创建: {output_path}")

    return ErrCode.SUCCESS


In [150]:
if __name__ == "__main__":
    print("PM订单处理脚本 - 排序优化版")
    print("=" * 60)
    print("自动检测并处理列名差异")
    print("按 Location 和 Model 排序")
    print("-" * 60)

    # 查找Excel文件
    target_file = find_excel_file()
    preprocess(target_file)



PM订单处理脚本 - 排序优化版
自动检测并处理列名差异
按 Location 和 Model 排序
------------------------------------------------------------
/Volumes/SSD 1TB/GEhealthcare/202508/Output
正在读取文件: /Volumes/SSD 1TB/GEhealthcare/202508/Preventive-Pending Report.xlsx
成功读取文件: 共 67 条记录


/Users/koen/miniconda3/envs/DemoEnv/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


TypeError: ord() expected a character, but string of length 2 found